In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Download video

In [ ]:
!pip install -q gdown
%cd {HOME}
!gdown '1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-'

In [ ]:
SOURCE_VIDEO_PATH = "/kaggle/input/testmoscow/KRA-2-9-2023-08-22-evening.mp4"

## Install YOLOv8

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## Install Roboflow Supervision

In [ ]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

## Load pre-trained YOLOv8 model

In [ ]:
MODEL = "yolov8x.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

## Predict and annotate single frame

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, bus and truck
selected_classes = [2, 5, 7]

In [ ]:
import supervision as sv
import numpy as np

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=1, text_scale=0.5)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _ in detections
]

# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

In [ ]:
import json
import numpy as np

def convert_coordinates_to_pixels(areas, zones, frame_resolution):
    pixel_coordinates = []

    for area in areas:
        polygon = np.array(area)
        polygon[:, 0] *= frame_resolution[0]
        polygon[:, 1] *= frame_resolution[1]
        pixel_coordinates.append(polygon)

    pixel_zone_coordinates = []

    for zone in zones:
        polygon = np.array(zone)
        polygon[:, 0] *= frame_resolution[0]
        polygon[:, 1] *= frame_resolution[1]
        pixel_zone_coordinates.append(polygon)

    return pixel_coordinates, pixel_zone_coordinates

# Загрузка данных из JSON-файла
json_file_path = "/kaggle/input/testmoscow/markup/jsons/KRA-2-9-2023-08-22-evening.json"  # Замените на путь к вашему JSON-файлу
with open(json_file_path, "r") as json_file:
    data = json.load(json_file)

# Разрешение кадра
frame_resolution = (1920, 1080)  # Замените на разрешение вашего видео

# Преобразование координат в пиксели
pixel_areas, pixel_zones = convert_coordinates_to_pixels(data["areas"], data["zones"], frame_resolution)

# Вывод координат в пикселях
print("Pixel Coordinates:")
for i, polygon in enumerate(pixel_areas):
    print(f"Area {i + 1} coordinates:\n{polygon}")

print("\nPixel Zone Coordinates:")
for i, polygon in enumerate(pixel_zones):
    print(f"Zone {i + 1} coordinates:\n{polygon}")

## Predict and annotate whole video

In [ ]:
# settings
LINE_START = sv.Point(632, 467)
LINE_END = sv.Point(646,510)

TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result-with-counter.mp4"

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [ ]:
# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=15)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=1, text_scale=0.5)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(thickness=1, trace_length=10)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=1, text_thickness=1, text_scale=0.5)


In [ ]:
# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
    f"#{tracker_id} {model.model.names[class_id]} {detection.confidence:0.2f}" +
    (f" @ ({int(x)}, {int(y)}) pixels" if len(detection) == 6 else "")
    for x, y, _, class_id, tracker_id, detection in detections
]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

In [ ]:

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, tracker_id
        in detections
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)